# Model Hypothesis
1. Insiders perspective:
An insider seeks profit in making insider buyback.
The more the buyback, the higher confidence he has – using his material insider information.

2. Our perspective:
The fundamentals: return, profitability, valuation, risk factors, etc.
The ‘better’ the fundamentals, the more convincing the insider ‘endorsement’ from part 1.

1+2: The ideal stocks (buy signals) appear when those two perspectives and evaluations match. 

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

In [38]:
# load the dataset into Python
df = pd.read_csv('INSIDER_CUSTM_3_26_csv.csv', delimiter=',')

# drop the top 1 blank row & bottom 10 N/A rows
df = df.drop(0)
df = df.drop(df.tail(11).index)
df
# in demand of data cleaning

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,LST Num Shs Purch_0M,LST Num Shs Purch_1M,LST Num Shs Purch_2M,LST Num Shs Purch_3M,LST Num Shs Purch_4M,LST Num Shs Purch_5M,...,Price /Sales CFY,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio
1,FLWS,"1-800-FLOWERS.COM, Inc. Class A",399.608000,65.617000,NaN,0.0,NaN,0.0,0.0,NaN,...,0.285009,21.249723,1.214968,0.406217,8.289439,17.830235,0.0,58.710781,7.524440,1.310658
2,VCXA,10x Capital Venture Acquisition Corp. II Class A,210.693390,27.321667,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.383362,NaN,NaN,NaN,0.0,0.000000,NaN,1.292346
3,TXG,10x Genomics Inc Class A,4879.304700,113.858684,0.0,0.0,NaN,0.0,0.0,NaN,...,8.034349,NaN,5.209733,7.380912,NaN,NaN,0.0,11.812203,-343.966387,4.849187
4,YI,"111, Inc. Sponsored ADR Class A",131.150280,83.317060,NaN,NaN,NaN,NaN,NaN,NaN,...,0.125350,NaN,NaN,0.155423,NaN,NaN,NaN,NaN,-27.110394,1.187411
5,YQ,"17 Education & Technology Group, Inc. Unsponso...",47.474472,49.405570,NaN,NaN,NaN,NaN,NaN,NaN,...,0.182325,NaN,0.421535,-0.296504,NaN,NaN,0.0,18.470182,NaN,1.965760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5098,CNET,ZW Data Action Technologies Inc,14.474381,6.609784,NaN,NaN,NaN,NaN,NaN,NaN,...,0.698295,NaN,1.623298,0.597404,NaN,NaN,0.0,11.017183,NaN,2.258021
5099,ZYME,Zymeworks Inc,525.138500,65.249184,0.0,NaN,2318900.0,0.0,0.0,0.0,...,1.243348,4.132275,1.028174,0.118201,0.137113,0.148916,NaN,5.706189,NaN,5.696272
5100,ZYNE,Zynerba Pharmaceuticals Inc,20.246380,39.259495,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,1.673961,NaN,NaN,NaN,0.0,0.793591,NaN,8.085458
5101,ZYXI,"Zynex, Inc.",402.612730,39.127000,0.0,0.0,0.0,0.0,0.0,0.0,...,3.441025,31.925637,7.719286,3.484112,19.916418,23.326121,0.0,40.626601,52.131818,3.298824


# Insider Buying attributes:
Sec Market Value (LOCAL), Com Shs to Calc Diluted EPS, LST Num Shs Purch_0M, LST Num Shs Purch_1M, LST Num Shs Purch_2M, LST Num Shs Purch_3M, LST Num Shs Purch_4M, LST Num Shs Purch_5M, LST Num Shs Purch_6M

# Return attributes:
Closing Price_latest, Closing Price_5D, Closing Price_1M, Closing Price_3M, Closing Price_6M, Closing Price_1Y

# Profitability attributes:
EBITDA Margin, EBIT Margin, Net Income Margin, ROA, ROE, ROIC, Asset Turnover

# Valuation attributes:
Price /Sales CFY, Price / Earns Ratio, Price / Book Value, Entrpr Value/ Sales, Entrpr Value/ EBITDA, Entrpr Value/ EBIT

# Management & Risk factors attributes:
Div Yield, Total Debt% Equity, EBIT Interest Coverage, Current Ratio

In [39]:
# Clean the dataset

# 1. drop rows that contain missing values of 'insider buying' & 'return'
df.dropna(subset=['Com Shs to Calc Diluted EPS'], inplace=True)
df.dropna(subset=['Close Price_latest', 'Close Price_1M', 'Close Price_3M', 'Close Price_6M', 'Close Price_1Y'], inplace=True)
# dropped 272 rows

# 2. drop rows that have closing price of zero
df = df[(df[['Close Price_latest', 'Close Price_1M', 'Close Price_3M', 'Close Price_6M', 'Close Price_1Y']] != 0).all(axis=1)]
# unchanged


In [40]:
# 3. convert the unit of 'millions'
df["Com Shs to Calc Diluted EPS"] = df["Com Shs to Calc Diluted EPS"] * 1000000
df["Sec Market Value (LOCAL)"] = df["Sec Market Value (LOCAL)"] * 1000000

In [41]:
# 4. replace with zero for missing 'LST Num Shs Purch'
df[['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 'LST Num Shs Purch_6M']] = df [['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 'LST Num Shs Purch_6M']].fillna(0)

# add them up to get ALL insider purchase within the last 6 months
df['Total Insider Purchases'] = df[['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 
                                    'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 
                                    'LST Num Shs Purch_6M']].sum(axis=1)

df = df.drop(['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 'LST Num Shs Purch_6M'], axis=1)


In [42]:
df['Total Insider Purchases'].describe()

count    4.830000e+03
mean     2.624854e+05
std      6.269918e+06
min     -7.000000e+03
25%      0.000000e+00
50%      0.000000e+00
75%      2.432500e+03
max      4.002133e+08
Name: Total Insider Purchases, dtype: float64

In [43]:
# 5. deal with untransparent stocks 
# drop the rows which have more than 70% missing values
threshold = len(df.columns) * 0.7  # set threshold to 70% of total columns
df = df.dropna(thresh=threshold)
df
# 5102 to 3686 rows

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
1,FLWS,"1-800-FLOWERS.COM, Inc. Class A",3.996080e+08,65617000.0,10.6000,10.1400,9.6100,6.1500,13.50,3.928522,...,21.249723,1.214968,0.406217,8.289439,17.830235,0.000000,58.710781,7.524440,1.310658,25000.0
3,TXG,10x Genomics Inc Class A,4.879305e+09,113858684.0,50.2300,47.0600,34.2400,28.9100,67.79,-19.407390,...,NaN,5.209733,7.380912,NaN,NaN,0.000000,11.812203,-343.966387,4.849187,0.0
9,SRCE,1st Source Corporation,1.109552e+09,24687324.0,44.9200,49.4200,54.2600,47.8300,48.58,NaN,...,10.875978,1.515298,4.238425,NaN,NaN,2.373328,39.404538,NaN,0.030762,1000.0
10,DIBS,"1stdibs.com, Inc.",1.472556e+08,38479437.0,3.7500,5.0000,5.0000,6.0800,8.19,-24.133530,...,NaN,1.347380,0.673363,NaN,NaN,0.000000,16.516464,-3232.818182,6.236583,0.0
11,XXII,"22nd Century Group, Inc.",1.615408e+08,192837000.0,0.7489,0.9258,0.8842,0.9696,2.41,-49.078670,...,NaN,2.064364,2.624728,NaN,NaN,0.000000,7.013285,-142.042493,3.426410,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5094,ZUMZ,Zumiez Inc.,3.434064e+08,19428000.0,17.6200,23.7000,21.0100,21.8800,39.74,3.160622,...,23.783135,1.232033,0.606312,2.673720,9.842831,0.000000,62.430032,NaN,2.333038,0.0
5095,ZUO,"Zuora, Inc. Class A",1.168405e+09,131441000.0,9.3100,8.6200,5.8000,7.3400,14.97,10.034270,...,NaN,NaN,2.303370,NaN,NaN,NaN,148.745091,-6.418291,2.131704,0.0
5097,ZWS,Zurn Elkay Water Solutions Corporation,3.494993e+09,153837000.0,19.8500,22.9700,21.4300,24.8800,35.50,22.055150,...,57.085020,2.316369,2.893082,20.704817,29.916347,0.945626,35.882353,4.553903,2.618389,0.0
5099,ZYME,Zymeworks Inc,5.251385e+08,65249184.0,8.2000,8.6200,8.0400,5.2700,7.00,-243.809100,...,4.132275,1.028174,0.118201,0.137113,0.148916,NaN,5.706189,NaN,5.696272,2318900.0


In [44]:
# drop the rows that have outlier valuations

# define the list of columns to clean
cols_to_clean = ['Price /Sales CFY', 'Price / Earns Ratio', 'Price / Book Value', 'Entrpr Value/ Sales', 'Entrpr Value/ EBITDA', 'Entrpr Value/ EBIT']

# loop through each column and drop the outliers
for col in cols_to_clean:
    q_low = df[col].quantile(0.001)
    q_high = df[col].quantile(0.999)
    df = df[(df[col] >= q_low) & (df[col] <= q_high)]
# 3686 to 1729 rows

In [45]:
df

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
1,FLWS,"1-800-FLOWERS.COM, Inc. Class A",3.996080e+08,65617000.0,10.60,10.14,9.61,6.15,13.50,3.928522,...,21.249723,1.214968,0.406217,8.289439,17.830235,0.000000,58.710781,7.524440,1.310658,25000.0
19,MMM,3M Company,5.567757e+10,567600000.0,101.14,107.80,120.14,112.99,150.46,24.811880,...,11.782391,4.473949,2.368973,13.591902,19.702085,4.969980,114.563239,8.178715,1.542371,0.0
32,AMRK,"A-Mark Precious Metals, Inc.",7.654299e+08,24329500.0,32.53,29.82,34.41,26.49,39.70,2.364319,...,5.920147,1.543159,0.162685,9.524190,11.860207,0.000000,134.759695,5.061159,1.411588,0.0
33,AOS,A. O. Smith Corporation,8.467280e+09,155779364.0,66.74,64.90,56.90,49.83,66.97,19.956220,...,37.832122,4.951360,2.347375,12.002065,13.412024,1.991614,21.559764,69.638298,1.748769,4000.0
34,ATEN,"A10 Networks, Inc.",1.109844e+09,77751000.0,14.98,14.80,17.02,13.00,13.93,20.809880,...,27.565059,7.113039,4.150875,18.888689,21.515291,1.262778,11.953904,NaN,2.121364,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,ZI,ZoomInfo Technologies Inc,9.371473e+09,403392458.0,23.18,25.19,28.80,40.04,56.41,44.233290,...,192.150606,5.355431,11.761700,47.365856,70.066304,0.000000,57.835197,3.848739,1.452768,0.0
5094,ZUMZ,Zumiez Inc.,3.434064e+08,19428000.0,17.62,23.70,21.01,21.88,39.74,3.160622,...,23.783135,1.232033,0.606312,2.673720,9.842831,0.000000,62.430032,NaN,2.333038,0.0
5097,ZWS,Zurn Elkay Water Solutions Corporation,3.494993e+09,153837000.0,19.85,22.97,21.43,24.88,35.50,22.055150,...,57.085020,2.316369,2.893082,20.704817,29.916347,0.945626,35.882353,4.553903,2.618389,0.0
5099,ZYME,Zymeworks Inc,5.251385e+08,65249184.0,8.20,8.62,8.04,5.27,7.00,-243.809100,...,4.132275,1.028174,0.118201,0.137113,0.148916,NaN,5.706189,NaN,5.696272,2318900.0


In [46]:
# drop the rows that have outlier profitability

# define the list of columns to clean
cols_to_clean = ['EBITDA Margin', 'EBIT Margin', 'Net Income Margin', 'ROA', 'ROE', 'ROIC', 'Asset Turnover']

# loop through each column and drop the outliers
'''
for col in cols_to_clean:
    q_low = df[col].quantile(0.001)
    q_high = df[col].quantile(0.9999)
    df = df[(df[col] >= q_low) & (df[col] <= q_high)]
'''
# 1729 to 864 rows

'\nfor col in cols_to_clean:\n    q_low = df[col].quantile(0.001)\n    q_high = df[col].quantile(0.9999)\n    df = df[(df[col] >= q_low) & (df[col] <= q_high)]\n'

In [47]:
# define the list of columns to clean
cols_to_clean = ['EBITDA Margin', 'EBIT Margin', 'Net Income Margin', 'ROA', 'ROE', 'ROIC', 'Asset Turnover']

# loop through each column and drop the outliers
for col in cols_to_clean:
    q_low = df[col].quantile(0.001)
    
    df = df[(df[col] >= q_low)]

In [48]:
df

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
19,MMM,3M Company,5.567757e+10,5.676000e+08,101.14,107.80,120.14,112.99,150.46,24.81188,...,11.782391,4.473949,2.368973,13.591902,19.702085,4.969980,114.563239,8.178715,1.542371,0.0
33,AOS,A. O. Smith Corporation,8.467280e+09,1.557794e+08,66.74,64.90,56.90,49.83,66.97,19.95622,...,37.832122,4.951360,2.347375,12.002065,13.412024,1.991614,21.559764,69.638298,1.748769,4000.0
39,ABT,Abbott Laboratories,1.704056e+11,1.764000e+09,98.05,100.46,108.18,100.68,118.95,25.70148,...,28.047721,5.200690,4.619718,17.240654,23.976508,1.748793,48.917843,26.546032,1.628511,0.0
40,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.33,48.39309,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
41,ABCM,Abcam PLC Sponsored ADR,2.976440e+09,2.289000e+08,12.98,14.26,15.45,14.63,18.04,33.54594,...,882.022472,6.056398,12.847576,58.726456,144.718764,0.000000,35.009907,10.370371,1.121481,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5081,ZBH,"Zimmer Biomet Holdings, Inc.",2.647650e+10,2.103000e+08,126.04,122.70,126.69,106.21,125.28,34.33426,...,115.877488,2.216875,4.663043,15.092388,26.712724,0.752941,49.223397,7.332115,1.877406,0.0
5086,ZIP,"ZipRecruiter, Inc. Class A",1.138849e+09,1.213980e+08,15.22,17.35,16.26,15.81,22.76,23.40767,...,32.418559,59.955450,2.192017,17.487152,20.205693,0.000000,1956.694619,3.411748,4.838882,0.0
5089,ZTS,"Zoetis, Inc. Class A",7.607883e+10,4.703850e+08,164.18,166.33,145.76,150.05,189.43,42.96776,...,32.608696,16.697430,9.094669,21.590310,25.019099,0.921187,184.631101,11.348837,2.370066,0.0
5090,ZM,"Zoom Video Communications, Inc. Class A",1.695982e+10,3.042314e+08,69.04,73.93,65.89,74.45,113.93,38.89104,...,220.005867,3.550524,3.980920,28.230689,70.861814,0.000000,1.554424,NaN,3.527555,0.0


In [49]:
# 6. drop the columns which have more than 40% missing values
# what metrics were dropped?
dropped_cols = []
for col in df.columns:
    if df[col].isna().sum() > len(df)*0.4:
        df.drop(col, axis=1, inplace=True)
        dropped_cols.append(col)

print("Dropped columns:", dropped_cols)
# dropped columns (metrics): 

Dropped columns: []


In [50]:
# Calculate the percentage of missing values in each column
missing_perc = df.isna().mean() * 100

# Get the column names where the percentage of missing values is greater than 40%
columns_to_drop = missing_perc[missing_perc > 40].index

# Drop the columns from the dataframe
df = df.drop(columns_to_drop, axis=1)

In [51]:
df

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
19,MMM,3M Company,5.567757e+10,5.676000e+08,101.14,107.80,120.14,112.99,150.46,24.81188,...,11.782391,4.473949,2.368973,13.591902,19.702085,4.969980,114.563239,8.178715,1.542371,0.0
33,AOS,A. O. Smith Corporation,8.467280e+09,1.557794e+08,66.74,64.90,56.90,49.83,66.97,19.95622,...,37.832122,4.951360,2.347375,12.002065,13.412024,1.991614,21.559764,69.638298,1.748769,4000.0
39,ABT,Abbott Laboratories,1.704056e+11,1.764000e+09,98.05,100.46,108.18,100.68,118.95,25.70148,...,28.047721,5.200690,4.619718,17.240654,23.976508,1.748793,48.917843,26.546032,1.628511,0.0
40,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.33,48.39309,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
41,ABCM,Abcam PLC Sponsored ADR,2.976440e+09,2.289000e+08,12.98,14.26,15.45,14.63,18.04,33.54594,...,882.022472,6.056398,12.847576,58.726456,144.718764,0.000000,35.009907,10.370371,1.121481,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5081,ZBH,"Zimmer Biomet Holdings, Inc.",2.647650e+10,2.103000e+08,126.04,122.70,126.69,106.21,125.28,34.33426,...,115.877488,2.216875,4.663043,15.092388,26.712724,0.752941,49.223397,7.332115,1.877406,0.0
5086,ZIP,"ZipRecruiter, Inc. Class A",1.138849e+09,1.213980e+08,15.22,17.35,16.26,15.81,22.76,23.40767,...,32.418559,59.955450,2.192017,17.487152,20.205693,0.000000,1956.694619,3.411748,4.838882,0.0
5089,ZTS,"Zoetis, Inc. Class A",7.607883e+10,4.703850e+08,164.18,166.33,145.76,150.05,189.43,42.96776,...,32.608696,16.697430,9.094669,21.590310,25.019099,0.921187,184.631101,11.348837,2.370066,0.0
5090,ZM,"Zoom Video Communications, Inc. Class A",1.695982e+10,3.042314e+08,69.04,73.93,65.89,74.45,113.93,38.89104,...,220.005867,3.550524,3.980920,28.230689,70.861814,0.000000,1.554424,NaN,3.527555,0.0


In [52]:
df.reset_index(drop=True, inplace=True)
df

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
0,MMM,3M Company,5.567757e+10,5.676000e+08,101.14,107.80,120.14,112.99,150.46,24.81188,...,11.782391,4.473949,2.368973,13.591902,19.702085,4.969980,114.563239,8.178715,1.542371,0.0
1,AOS,A. O. Smith Corporation,8.467280e+09,1.557794e+08,66.74,64.90,56.90,49.83,66.97,19.95622,...,37.832122,4.951360,2.347375,12.002065,13.412024,1.991614,21.559764,69.638298,1.748769,4000.0
2,ABT,Abbott Laboratories,1.704056e+11,1.764000e+09,98.05,100.46,108.18,100.68,118.95,25.70148,...,28.047721,5.200690,4.619718,17.240654,23.976508,1.748793,48.917843,26.546032,1.628511,0.0
3,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.33,48.39309,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
4,ABCM,Abcam PLC Sponsored ADR,2.976440e+09,2.289000e+08,12.98,14.26,15.45,14.63,18.04,33.54594,...,882.022472,6.056398,12.847576,58.726456,144.718764,0.000000,35.009907,10.370371,1.121481,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",2.647650e+10,2.103000e+08,126.04,122.70,126.69,106.21,125.28,34.33426,...,115.877488,2.216875,4.663043,15.092388,26.712724,0.752941,49.223397,7.332115,1.877406,0.0
865,ZIP,"ZipRecruiter, Inc. Class A",1.138849e+09,1.213980e+08,15.22,17.35,16.26,15.81,22.76,23.40767,...,32.418559,59.955450,2.192017,17.487152,20.205693,0.000000,1956.694619,3.411748,4.838882,0.0
866,ZTS,"Zoetis, Inc. Class A",7.607883e+10,4.703850e+08,164.18,166.33,145.76,150.05,189.43,42.96776,...,32.608696,16.697430,9.094669,21.590310,25.019099,0.921187,184.631101,11.348837,2.370066,0.0
867,ZM,"Zoom Video Communications, Inc. Class A",1.695982e+10,3.042314e+08,69.04,73.93,65.89,74.45,113.93,38.89104,...,220.005867,3.550524,3.980920,28.230689,70.861814,0.000000,1.554424,NaN,3.527555,0.0


In [53]:
# 7.* impute missing values with median
imputer = SimpleImputer(strategy='median')
cols_to_impute = df.columns[2:]
df_imputed = pd.DataFrame(imputer.fit_transform(df[cols_to_impute]), columns=cols_to_impute)

# add back the columns of symbol & name
df_imputed[['Symbol', 'Name']] = df[['Symbol', 'Name']]

# Move Symbol and Name back to the leftmost position & check the resulting dataframe
cols = ['Symbol', 'Name'] + [col for col in df_imputed.columns if col not in ['Symbol', 'Name']]
df_imp = df_imputed[cols]
df_imp


,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
0,MMM,3M Company,5.567757e+10,5.676000e+08,101.14,107.80,120.14,112.99,150.46,24.81188,...,11.782391,4.473949,2.368973,13.591902,19.702085,4.969980,114.563239,8.178715,1.542371,0.0
1,AOS,A. O. Smith Corporation,8.467280e+09,1.557794e+08,66.74,64.90,56.90,49.83,66.97,19.95622,...,37.832122,4.951360,2.347375,12.002065,13.412024,1.991614,21.559764,69.638298,1.748769,4000.0
2,ABT,Abbott Laboratories,1.704056e+11,1.764000e+09,98.05,100.46,108.18,100.68,118.95,25.70148,...,28.047721,5.200690,4.619718,17.240654,23.976508,1.748793,48.917843,26.546032,1.628511,0.0
3,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.33,48.39309,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
4,ABCM,Abcam PLC Sponsored ADR,2.976440e+09,2.289000e+08,12.98,14.26,15.45,14.63,18.04,33.54594,...,882.022472,6.056398,12.847576,58.726456,144.718764,0.000000,35.009907,10.370371,1.121481,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",2.647650e+10,2.103000e+08,126.04,122.70,126.69,106.21,125.28,34.33426,...,115.877488,2.216875,4.663043,15.092388,26.712724,0.752941,49.223397,7.332115,1.877406,0.0
865,ZIP,"ZipRecruiter, Inc. Class A",1.138849e+09,1.213980e+08,15.22,17.35,16.26,15.81,22.76,23.40767,...,32.418559,59.955450,2.192017,17.487152,20.205693,0.000000,1956.694619,3.411748,4.838882,0.0
866,ZTS,"Zoetis, Inc. Class A",7.607883e+10,4.703850e+08,164.18,166.33,145.76,150.05,189.43,42.96776,...,32.608696,16.697430,9.094669,21.590310,25.019099,0.921187,184.631101,11.348837,2.370066,0.0
867,ZM,"Zoom Video Communications, Inc. Class A",1.695982e+10,3.042314e+08,69.04,73.93,65.89,74.45,113.93,38.89104,...,220.005867,3.550524,3.980920,28.230689,70.861814,0.000000,1.554424,7.808558,3.527555,0.0


In [54]:
df_imp.describe()

,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,EBIT Margin,Net Income Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
count,8.690000e+02,8.690000e+02,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,...,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,869.000000,8.690000e+02
mean,3.318546e+10,3.595120e+08,106.378441,109.029833,103.109969,95.430762,115.655953,30.014207,21.415101,13.821599,...,43.534045,5.638180,4.028703,14.316967,23.830760,1.956509,169.388866,60.351610,1.883659,7.179073e+04
std,1.271113e+11,9.410353e+08,232.281936,222.961970,201.501123,181.413775,215.910629,22.532966,17.512429,12.226260,...,175.723125,12.556074,4.127693,13.237136,31.147655,2.810959,369.827741,1066.498282,1.499038,8.402508e+05
min,3.947590e+07,2.889991e+06,1.310000,1.470000,2.540000,4.550000,5.570000,-0.160335,-12.500360,0.158631,...,0.413225,0.324731,0.122537,1.080138,1.210528,0.000000,0.000000,0.383506,0.332947,0.000000e+00
25%,2.976440e+09,6.457900e+07,26.910000,28.640000,28.380000,26.380000,33.110000,14.575710,10.143180,5.551376,...,12.377991,1.657370,1.311132,7.384254,10.871730,0.000000,50.727489,3.694915,1.145546,0.000000e+00
50%,7.302719e+09,1.393063e+08,59.560000,63.150000,59.830000,56.300000,66.860000,23.706440,17.368140,10.266278,...,20.664215,2.702466,2.567181,12.308956,17.783574,1.204365,87.871975,7.808558,1.527531,0.000000e+00
75%,2.478352e+10,3.164590e+08,124.620000,125.130000,120.140000,111.260000,135.710000,38.511310,28.075340,18.049683,...,32.653679,5.051855,5.037833,17.236082,26.307501,2.766861,157.574905,16.196179,2.097131,0.000000e+00
max,2.535467e+12,1.632582e+10,5413.000000,5132.470000,4645.000000,4057.670000,4668.040000,252.649900,252.649900,103.992766,...,4474.012462,205.990600,41.787142,211.387959,566.699636,33.801353,6895.756441,31236.439020,18.583703,1.967395e+07


In [55]:
# create a new DataFrame to record our attribute-group metrics
df_2 = pd.DataFrame({
    'Symbol': df['Symbol'],
    'Name': df['Name'],
    'Insider Buying %' : df_imp['Total Insider Purchases'] / df_imp['Com Shs to Calc Diluted EPS'] * 100
})

df_2

,Symbol,Name,Insider Buying %
0,MMM,3M Company,0.000000
1,AOS,A. O. Smith Corporation,0.002568
2,ABT,Abbott Laboratories,0.000000
3,ABBV,"AbbVie, Inc.",0.000000
4,ABCM,Abcam PLC Sponsored ADR,0.000000
...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",0.000000
865,ZIP,"ZipRecruiter, Inc. Class A",0.000000
866,ZTS,"Zoetis, Inc. Class A",0.000000
867,ZM,"Zoom Video Communications, Inc. Class A",0.000000


In [56]:
# calculate returns for different time periods
df_imp['Return_1M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_1M']) / df['Close Price_1M']
df_imp['Return_3M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_3M']) / df['Close Price_3M']
df_imp['Return_6M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_6M']) / df['Close Price_6M']
df_imp['Return_1Y'] = (df_imp['Close Price_latest'] - df_imp['Close Price_1Y']) / df['Close Price_1Y']
df_imp

C:\Users\l3047\AppData\Local\Temp\ipykernel_18764\4052276547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imp['Return_1M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_1M']) / df['Close Price_1M']
C:\Users\l3047\AppData\Local\Temp\ipykernel_18764\4052276547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imp['Return_3M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_3M']) / df['Close Price_3M']
C:\Users\l3047\AppData\Local\Temp\ipykernel_18764\4052276547.py:4: SettingWithCo

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases,Return_1M,Return_3M,Return_6M,Return_1Y
0,MMM,3M Company,5.567757e+10,5.676000e+08,101.14,107.80,120.14,112.99,150.46,24.81188,...,19.702085,4.969980,114.563239,8.178715,1.542371,0.0,-0.061781,-0.158149,-0.104877,-0.327795
1,AOS,A. O. Smith Corporation,8.467280e+09,1.557794e+08,66.74,64.90,56.90,49.83,66.97,19.95622,...,13.412024,1.991614,21.559764,69.638298,1.748769,4000.0,0.028351,0.172935,0.339354,-0.003434
2,ABT,Abbott Laboratories,1.704056e+11,1.764000e+09,98.05,100.46,108.18,100.68,118.95,25.70148,...,23.976508,1.748793,48.917843,26.546032,1.628511,0.0,-0.023990,-0.093640,-0.026122,-0.175704
3,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.33,48.39309,...,15.105636,3.533197,372.035470,9.634271,0.963606,0.0,0.034772,-0.031147,0.104572,-0.020517
4,ABCM,Abcam PLC Sponsored ADR,2.976440e+09,2.289000e+08,12.98,14.26,15.45,14.63,18.04,33.54594,...,144.718764,0.000000,35.009907,10.370371,1.121481,0.0,-0.089762,-0.159871,-0.112782,-0.280488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",2.647650e+10,2.103000e+08,126.04,122.70,126.69,106.21,125.28,34.33426,...,26.712724,0.752941,49.223397,7.332115,1.877406,0.0,0.027221,-0.005131,0.186706,0.006066
865,ZIP,"ZipRecruiter, Inc. Class A",1.138849e+09,1.213980e+08,15.22,17.35,16.26,15.81,22.76,23.40767,...,20.205693,0.000000,1956.694619,3.411748,4.838882,0.0,-0.122767,-0.063961,-0.037318,-0.331283
866,ZTS,"Zoetis, Inc. Class A",7.607883e+10,4.703850e+08,164.18,166.33,145.76,150.05,189.43,42.96776,...,25.019099,0.921187,184.631101,11.348837,2.370066,0.0,-0.012926,0.126372,0.094169,-0.133295
867,ZM,"Zoom Video Communications, Inc. Class A",1.695982e+10,3.042314e+08,69.04,73.93,65.89,74.45,113.93,38.89104,...,70.861814,0.000000,1.554424,7.808558,3.527555,0.0,-0.066144,0.047807,-0.072666,-0.394014


# Return

In [57]:
# select the return attributes
return_df = df_imp[['Return_1M', 'Return_3M', 'Return_6M', 'Return_1Y']]

# standardize the data
scaler = StandardScaler()
return_df_scaled = scaler.fit_transform(return_df)
pca = PCA(n_components=1)

# perform PCA on the standardized data
pca = PCA(n_components=1)
return_pca = pca.fit_transform(return_df_scaled)

df_2['Return'] = return_pca.flatten()
df_2

,Symbol,Name,Insider Buying %,Return
0,MMM,3M Company,0.000000,-1.390680
1,AOS,A. O. Smith Corporation,0.002568,1.837912
2,ABT,Abbott Laboratories,0.000000,-0.512445
3,ABBV,"AbbVie, Inc.",0.000000,0.600434
4,ABCM,Abcam PLC Sponsored ADR,0.000000,-1.482386
...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",0.000000,0.888255
865,ZIP,"ZipRecruiter, Inc. Class A",0.000000,-1.249933
866,ZTS,"Zoetis, Inc. Class A",0.000000,0.648456
867,ZM,"Zoom Video Communications, Inc. Class A",0.000000,-0.761540


# Profitability

In [58]:
# select the profitability metrics
profitability_df = df_imp[['EBITDA Margin', 'EBIT Margin', 'Net Income Margin', 'ROA', 'ROE','ROIC', 'Asset Turnover']]

# standardize the data
scaler = StandardScaler()
profitability_df_scaled = scaler.fit_transform(profitability_df)

# perform PCA on the standardized data
pca = PCA(n_components=1)
profitability_pca = pca.fit_transform(profitability_df_scaled)

df_2['Profitability'] = profitability_pca.flatten()
df_2

,Symbol,Name,Insider Buying %,Return,Profitability
0,MMM,3M Company,0.000000,-1.390680,-0.048293
1,AOS,A. O. Smith Corporation,0.002568,1.837912,-0.870505
2,ABT,Abbott Laboratories,0.000000,-0.512445,0.168202
3,ABBV,"AbbVie, Inc.",0.000000,0.600434,1.653139
4,ABCM,Abcam PLC Sponsored ADR,0.000000,-1.482386,0.008200
...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",0.000000,0.888255,0.251236
865,ZIP,"ZipRecruiter, Inc. Class A",0.000000,-1.249933,-0.710111
866,ZTS,"Zoetis, Inc. Class A",0.000000,0.648456,1.431559
867,ZM,"Zoom Video Communications, Inc. Class A",0.000000,-0.761540,0.447042


# Valuation 

In [59]:
# select the valuation metrics
valuation_df = df_imp[['Price /Sales CFY', 'Price / Earns Ratio','Price / Book Value', 'Entrpr Value/ Sales', 'Entrpr Value/ EBITDA', 'Entrpr Value/ EBIT']]

# standardize the data
scaler = StandardScaler()
valuation_df_scaled = scaler.fit_transform(valuation_df)

# perform PCA on the standardized data
pca = PCA(n_components=1)
valuation_pca = pca.fit_transform(valuation_df_scaled)

df_2['Valuation'] = valuation_pca.flatten()
df_2

,Symbol,Name,Insider Buying %,Return,Profitability,Valuation
0,MMM,3M Company,0.000000,-1.390680,-0.048293,-0.486418
1,AOS,A. O. Smith Corporation,0.002568,1.837912,-0.870505,-0.533664
2,ABT,Abbott Laboratories,0.000000,-0.512445,0.168202,0.311572
3,ABBV,"AbbVie, Inc.",0.000000,0.600434,1.653139,0.283475
4,ABCM,Abcam PLC Sponsored ADR,0.000000,-1.482386,0.008200,7.280843
...,...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",0.000000,0.888255,0.251236,0.333858
865,ZIP,"ZipRecruiter, Inc. Class A",0.000000,-1.249933,-0.710111,0.451222
866,ZTS,"Zoetis, Inc. Class A",0.000000,0.648456,1.431559,1.652969
867,ZM,"Zoom Video Communications, Inc. Class A",0.000000,-0.761540,0.447042,1.816349


In [60]:
df_2['Valuation_inver'] = 28 - df_2['Valuation']
df_2

,Symbol,Name,Insider Buying %,Return,Profitability,Valuation,Valuation_inver
0,MMM,3M Company,0.000000,-1.390680,-0.048293,-0.486418,28.486418
1,AOS,A. O. Smith Corporation,0.002568,1.837912,-0.870505,-0.533664,28.533664
2,ABT,Abbott Laboratories,0.000000,-0.512445,0.168202,0.311572,27.688428
3,ABBV,"AbbVie, Inc.",0.000000,0.600434,1.653139,0.283475,27.716525
4,ABCM,Abcam PLC Sponsored ADR,0.000000,-1.482386,0.008200,7.280843,20.719157
...,...,...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",0.000000,0.888255,0.251236,0.333858,27.666142
865,ZIP,"ZipRecruiter, Inc. Class A",0.000000,-1.249933,-0.710111,0.451222,27.548778
866,ZTS,"Zoetis, Inc. Class A",0.000000,0.648456,1.431559,1.652969,26.347031
867,ZM,"Zoom Video Communications, Inc. Class A",0.000000,-0.761540,0.447042,1.816349,26.183651


# Safety

In [61]:
# select the safety metrics
safety_df = df_imp[['Div Yield', 'EBIT Interest Coverage', 'Current Ratio']]

# standardize the data
scaler = StandardScaler()
safety_df_scaled = scaler.fit_transform(safety_df)

# perform PCA on the standardized data
pca = PCA(n_components=1)
safety_pca = pca.fit_transform(safety_df_scaled)

df_2['Safety'] = safety_pca.flatten()
df_2

,Symbol,Name,Insider Buying %,Return,Profitability,Valuation,Valuation_inver,Safety
0,MMM,3M Company,0.000000,-1.390680,-0.048293,-0.486418,28.486418,-0.719834
1,AOS,A. O. Smith Corporation,0.002568,1.837912,-0.870505,-0.533664,28.533664,-0.063003
2,ABT,Abbott Laboratories,0.000000,-0.512445,0.168202,0.311572,27.688428,-0.095844
3,ABBV,"AbbVie, Inc.",0.000000,0.600434,1.653139,0.283475,27.716525,-0.725661
4,ABCM,Abcam PLC Sponsored ADR,0.000000,-1.482386,0.008200,7.280843,20.719157,-0.022086
...,...,...,...,...,...,...,...,...
864,ZBH,"Zimmer Biomet Holdings, Inc.",0.000000,0.888255,0.251236,0.333858,27.666142,0.185753
865,ZIP,"ZipRecruiter, Inc. Class A",0.000000,-1.249933,-0.710111,0.451222,27.548778,1.665662
866,ZTS,"Zoetis, Inc. Class A",0.000000,0.648456,1.431559,1.652969,26.347031,0.381828
867,ZM,"Zoom Video Communications, Inc. Class A",0.000000,-0.761540,0.447042,1.816349,26.183651,1.071273


In [62]:
df_2.describe()

,Insider Buying %,Return,Profitability,Valuation,Valuation_inver,Safety
count,869.000000,8.690000e+02,8.690000e+02,8.690000e+02,869.000000,8.690000e+02
mean,0.048659,-1.584208e-17,-8.432074e-18,-2.759588e-17,28.000000,-2.341178e-17
std,0.536138,1.637052e+00,1.690800e+00,1.836588e+00,1.836588,1.109318e+00
min,0.000000,-6.630035e+00,-4.657330e+00,-1.780347e+00,0.255595,-5.771584e+00
25%,0.000000,-9.210980e-01,-1.112785e+00,-1.035804e+00,27.554813,-4.493316e-01
50%,0.000000,-6.321626e-02,-2.797065e-01,-4.529294e-01,28.452929,-6.791840e-02
75%,0.000000,9.424435e-01,8.140261e-01,4.451869e-01,29.035804,3.035739e-01
max,10.465774,8.330588e+00,1.287445e+01,2.774441e+01,29.780347,1.892670e+01


# Final Factor Score Calculation

In [63]:
# assign weights to each dimension
w_insider_buying = 0.3
w_return = 0.2
w_profitability = 0.2
w_valuation_inver = 0.2
w_safety = 0.1

# calculate the weighted score for each stock
df_2['Weighted Score'] = (
    w_insider_buying * df_2['Insider Buying %'] +
    w_return * df_2['Return'] +
    w_profitability * df_2['Profitability'] +
    w_valuation_inver * df_2['Valuation_inver'] +
    w_safety * df_2['Safety']
    
)

# sort the stocks by weighted score in descending order
df_2 = df_2.sort_values(by=['Weighted Score'], ascending=False)

# select the top 10 stocks
top_10 = df_2.head(10)

top_10

,Symbol,Name,Insider Buying %,Return,Profitability,Valuation,Valuation_inver,Safety,Weighted Score
751,SNX,TD SYNNEX Corporation,10.465774,-0.179794,-2.711581,-1.335210,29.335210,-0.176269,8.410872
512,MTH,Meritage Homes Corporation,0.000000,3.491797,-0.607593,-1.582326,29.582326,18.926695,8.385975
222,PLAY,"Dave & Buster's Entertainment, Inc.",8.658757,-0.431260,-0.590260,-0.914324,28.914324,-0.322832,8.143905
355,GRBK,"Green Brick Partners, Inc.",0.000000,5.006145,-0.330342,-1.459355,29.459355,3.451792,7.172211
693,STNG,Scorpio Tankers Inc.,0.000000,3.510444,3.287565,-0.975748,28.975748,-0.006967,7.154055
414,INSW,"International Seaways, Inc.",0.000000,3.183814,3.345615,-0.994722,28.994722,0.398978,7.144728
506,MEDP,"Medpace Holdings, Inc.",5.757952,0.370128,-0.566936,0.798249,27.201751,-0.300466,7.098327
392,HHC,Howard Hughes Corporation,4.629329,0.105263,0.064434,-0.158156,28.158156,0.159111,7.070280
778,TPH,"Tri Pointe Homes, Inc.",0.000000,3.387167,-0.604390,-1.590403,29.590403,5.586819,7.033318
259,ELF,"e.l.f. Beauty, Inc.",0.000000,8.330588,-0.781231,0.922647,27.077353,0.819308,7.007273


In [64]:
ticker_list = top_10['Symbol'].tolist()
print(ticker_list)

['SNX', 'MTH', 'PLAY', 'GRBK', 'STNG', 'INSW', 'MEDP', 'HHC', 'TPH', 'ELF']
